# atmodeller

## Tutorial 5: Interior (include Si in the atmosphere)

Import the required functionality and activate the debug logger.

In [ ]:
from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import  MassConstraint, SystemConstraints, IronWustiteBufferConstraintHirschmann, FugacityConstraint, TotalPressureConstraint
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.interfaces import GasSpecies, LiquidSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2
from atmodeller.utilities import earth_oceans_to_kg
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# debug_file_logger()
#debug_logger()

### 1a. H2O-H2-O2-SiO-SiH4-SiO2l system

We define a list of the species we wish to include in the interior-atmosphere system

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))#, solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))


Create an interior-atmosphere system using H-O-Si for a sub-Neptune (Planet mass = 11 M_Earth, Interior radius = 1.7 R_Earth)

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

To solve the system, we provide a constraint of the H2O and O2 fugacity in bar and assume the actvity of SiO2(l)=1.

In [ ]:
fH2 = 10000 # bar
ratio_H2_H2O = 1000

constraints: SystemConstraints = SystemConstraints(
    [
        FugacityConstraint(species='H2', value=fH2), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
        FugacityConstraint(species='H2O', value=fH2/ratio_H2_H2O), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
        FugacityConstraint(species='O2Si', value=1),  # This constraints simulates activity of 1
        # IronWustiteBufferConstraintHirschmann(log10_shift=-4), # This constraint sets a low fO2 simulating a sub-Neptune surface
    ])


In [ ]:
interior_atmosphere.solve(constraints)

You can access the solution directly using:

In [ ]:
interior_atmosphere.solution_dict

You can access more information about the solution using:

In [ ]:
interior_atmosphere.output

### 1b. H2/H2O ratio variation: H2O-H2-O2-SiO-SiH4-SiO2l system 

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))#, solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))


In [ ]:
planet = Planet(surface_temperature=3400, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
fH2 = 100000 # bar
ratio_H2_H2Os = np.logspace(-2, 4, num=100) 
aSiO2l = 1

h_kg: float = earth_oceans_to_kg(1)

out = []

for ratio_H2_H2O in ratio_H2_H2Os:

    constraints: SystemConstraints = SystemConstraints(
        [
            #MassConstraint(species="H", value=h_kg),
            FugacityConstraint(species='H2', value=fH2), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
            FugacityConstraint(species='H2O', value=fH2/ratio_H2_H2O), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
            FugacityConstraint(species='O2Si', value=aSiO2l),  # This constraints simulates activity of 1
            #IronWustiteBufferConstraintHirschmann(log10_shift=-2* np.log10(ratio_H2_H2O)), # This constraint sets a low fO2 simulating a sub-Neptune surface
            #TotalPressureConstraint(species='None', value=fH2),
        ])

    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    interior_atmosphere.solve(constraints, factor=1)
    outi = interior_atmosphere.solution_dict

    # Include the parameters in the output.
    #outi['fH2'] = fH2
    outi['ratio_H2_H2O'] = ratio_H2_H2O
    outi['aSiO2l'] = aSiO2l

    out.append(outi)

filename = "atmodeller_SiOH_tutorial5.csv"
df = pd.DataFrame(out)
df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv(filename) 
ratio = df['ratio_H2_H2O']
H2 = df['H2']
H2O = df['H2O']
SiH4 = df['H4Si']
SiO = df['OSi']
O2 = df['O2']
tot = H2 + H2O + SiH4 + SiO + O2

fig, ax1 = plt.subplots(1, figsize=(5,3))

ax1.axvspan(1e-2, 1e0, alpha=0.2, color='blue')
ax1.axvspan(1e0, 1e2, alpha=0.2, color='green')
ax1.axvspan(1e2, 1e4, alpha=0.2, color='red')

ax1.text(2e-2, 1e-3, 'Steam Worlds', fontsize=10)
ax1.text(1, 1e-5, 'Hydrogen Worlds', fontsize=10)
ax1.text(1.5e2, 1e-7, 'Silane Worlds', fontsize=10)

ax1.plot(ratio, H2/tot, color='orange', lw=2, label=r'H$_2$')
ax1.plot(ratio, H2O/tot, color='blue', lw=2, label=r'H$_2$O')
ax1.plot(ratio, SiH4/tot, color='red', lw=2, label=r'SiH$_4$')
ax1.plot(ratio, SiO/tot, color='brown', lw=2, label=r'SiO')

ax1.set_title('Atmospheric Composition at the Surface')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$P_{\rm H_2}/P_{\rm H_2O}$', fontsize=16)
ax1.set_ylabel(r'Mixing Ratios', fontsize=16)

ax1.legend(ncol=2)
plt.savefig('atmodeller_SiOH_fH2fH2O.pdf', bbox_inches='tight')
plt.show()

### 1c. H2/H2O ratio variation: H2O-H2-O2-SiO-SiH4-SiO2l system 

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))#, solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='O2Si'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(GasSpecies(chemical_formula='HSi'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))


In [ ]:
planet = Planet(surface_temperature=3400, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
fH2 = 100000 # bar
ratio_H2_H2Os = np.logspace(-2, 4, num=100) 
aSiO2l = 1

out = []

for ratio_H2_H2O in ratio_H2_H2Os:

    constraints: SystemConstraints = SystemConstraints(
        [
            FugacityConstraint(species='H2', value=fH2), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
            FugacityConstraint(species='H2O', value=fH2/ratio_H2_H2O), # This constraint sets surface pressure of approx 10 kbar (1 GPa)
            FugacityConstraint(species='O2Si', value=aSiO2l),  # This constraints simulates activity of 1
            # IronWustiteBufferConstraintHirschmann(log10_shift=-4), # This constraint sets a low fO2 simulating a sub-Neptune surface
        ])

    # Recall that changing attributes on the planet 'object' will be 'seen' by interior_atmosphere.
    interior_atmosphere.solve(constraints, factor=0.1)
    outi = interior_atmosphere.solution_dict

    # Include the parameters in the output.
    outi['fH2'] = fH2
    outi['ratio_H2_H2O'] = ratio_H2_H2O
    outi['aSiO2l'] = aSiO2l

    out.append(outi)

filename = "atmodeller_SiOH_tutorial5.csv"
df = pd.DataFrame(out)
df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv(filename) 
ratio = df['ratio_H2_H2O']
H2 = df['H2']
H2O = df['H2O']
SiH4 = df['H4Si']
SiO = df['OSi']
SiO2 = df['O2Si']
SiH = df['HSi']
O2 = df['O2']
tot = H2 + H2O + SiH4 + SiO + O2

fig, ax1 = plt.subplots(1, figsize=(5,3))

ax1.axvspan(1e-2, 1e0, alpha=0.2, color='blue')
ax1.axvspan(1e0, 1e2, alpha=0.2, color='green')
ax1.axvspan(1e2, 1e4, alpha=0.2, color='red')

ax1.text(2e-2, 1e-3, 'Steam Worlds', fontsize=10)
ax1.text(1, 1e-5, 'Hydrogen Worlds', fontsize=10)
ax1.text(1.5e2, 1e-7, 'Silane Worlds', fontsize=10)

ax1.plot(ratio, H2/tot, color='orange', lw=2, label=r'H$_2$')
ax1.plot(ratio, H2O/tot, color='blue', lw=2, label=r'H$_2$O')
ax1.plot(ratio, SiH4/tot, color='red', lw=2, label=r'SiH$_4$')
ax1.plot(ratio, SiO2/tot, color='gray', lw=2, label=r'SiO2')
ax1.plot(ratio, SiO/tot, color='brown', lw=2, label=r'SiO')
ax1.plot(ratio, SiH/tot, color='green', lw=2, label=r'SiH')

ax1.set_title('Atmospheric Composition at the Surface')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$P_{\rm H_2}/P_{\rm H_2O}$', fontsize=16)
ax1.set_ylabel(r'Mixing Ratios', fontsize=16)

ax1.legend(ncol=2)
plt.savefig('atmodeller_SiOHfull_fH2fH2O.pdf', bbox_inches='tight')
plt.show()

### 2. H2O-H2-O2-SiO-SiH4-SiO2l-CH4-CO-CO2 system

We now extend the species list to additionally include C-species:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

An additional constraint is required to solve the system

In [ ]:
constraints: SystemConstraints = SystemConstraints(
    [
        FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
        FugacityConstraint(species='O2Si', value=1),  # This constraints simulates activity of 1
        FugacityConstraint(species='CO', value=1),  # This is an arbitrary constraint
        IronWustiteBufferConstraintHirschmann(log10_shift=-4),  # This constraint sets a low fO2 simulating a sub-Neptune surface
    ])


In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

### 3. System with mixed constraints

A typical use case is to define an interior-atmosphere system with a combination of fugacity and mass constraints. We define the same species as before:

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O'))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2'))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

Now we define the constraints, and in this case we want to constrain the total mass of C and H in the system that can partition between the various reservoirs.

In [ ]:
number_of_earth_oceans: float = 11 # 11x to account for mass increase by 11x
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans) # note: this constraint is not used below
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
    # MassConstraint(species="H", value=mass_H), # Mass constraint of H
    MassConstraint(species="C", value=mass_C), # Mass constraint of C
    FugacityConstraint(species='O2Si', value=1), # Simulates an activity of unity
    IronWustiteBufferConstraintHirschmann(log10_shift=-4) # Sets a low fO2 for a sub-Neptune surface
])

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

Solution

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict

### 4. System with mixed constraints and H2O-CO2 solubilities

Include H2O and CO2 solubilities

In [ ]:
species: Species = Species()
species.append(GasSpecies(chemical_formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(chemical_formula='H2'))
species.append(GasSpecies(chemical_formula='O2'))
species.append(GasSpecies(chemical_formula='CH4'))
species.append(GasSpecies(chemical_formula='CO'))
species.append(GasSpecies(chemical_formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(chemical_formula='OSi'))
species.append(GasSpecies(chemical_formula='H4Si'))
species.append(LiquidSpecies(chemical_formula="O2Si",
                             name_in_thermodynamic_data="O2Si(l)",
                             #thermodynamic_class=thermodynamic_data,
                            ))

species

In [ ]:
number_of_earth_oceans: float = 11 # 11x to account for mass increase by 11x
# C/H ratio by mass.
ch_ratio: float = 1

mass_H: float = earth_oceans_to_kg(number_of_earth_oceans) # note: this constraint is not used below
mass_C: float = ch_ratio * mass_H

constraints: SystemConstraints = SystemConstraints([
    FugacityConstraint(species='H2', value=10000), # This constraint sets surface pressure of 10 kbar (1 GPa)
    # MassConstraint(species="H", value=mass_H), # Mass constraint of H
    MassConstraint(species="C", value=mass_C), # Mass constraint of C
    FugacityConstraint(species='O2Si', value=1), # Simulates an activity of unity
    IronWustiteBufferConstraintHirschmann(log10_shift=-4) # Sets a low fO2 for a sub-Neptune surface
])

In [ ]:
planet = Planet(surface_temperature=3000, mantle_mass=11*4.208261222595111e24, surface_radius=1.7*6371000)
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)

In [ ]:
interior_atmosphere.solve(constraints)
interior_atmosphere.solution_dict